# Digit recognition Notebook
This notebook should outline how the digit recognition script included in this project should work. It should outline how a Convolutional Neural Network works. It should also cover how to run the script and the different functions that it performs.

The script was trained using the MNIST dataset that was covered in a previous notebook. MNIST is a dataset that consists of 60,000 training images and labels with hand written digits in the range of 0-9 that was taken from the NIST dataset. 

## Running the script
To run the script you will need to clone the git repository from [here](https://github.com/lanodburke/Emerging-Technologies-Project.git).

This script takes in command line arguments which are outlined below: 
- -h (help): will print some instructions describing the different parameters
- -b (build): this will build the keras model that will be used to run the digit recognition
- -t (test): this will take in an input file as an argument such as a PNG, JPG or any other image format

### Build the model 
Builds the model and saves it with the .h5 file format
```zsh
python digitrec.py -b
```

### Test the model
Run the saved model and make a prediction on the image passed in as an argument
```zsh
python digitrec.py -t image.png
```

## Convolutional Neural Network
> Convolutional Neural Networks have a different architecture than regular Neural Networks. Regular Neural Networks transform an input by putting it through a series of hidden layers. Every layer is made up of a set of neurons, where each layer is fully connected to all neurons in the layer before. Finally, there is a last fully-connected layer — the output layer — that represent the predictions.

> Convolutional Neural Networks are a bit different. First of all, the layers are organised in 3 dimensions: width, height and depth. Further, the neurons in one layer do not connect to all the neurons in the next layer but only to a small region of it. Lastly, the final output will be reduced to a single vector of probability scores, organized along the depth dimension.

### Neural Network Architecture 
![alt text](http://cs231n.github.io/assets/nn1/neural_net2.jpeg "Neural Network")

### Convolutional Neural Network Architecture
![alt text](http://cs231n.github.io/assets/cnn/cnn.jpeg "Convolutional Neural Network")


Convolutional Nerual networks have two parts: 
- Hidden layers/Feature extraction (Common in most image classifiers)
- Classification

### Feature Extraction
To understand feature extraction we first have to look at what is an image feature. 

An image feature can be described as a section of an image that is consistent with images of the same object. For example take an image of a cat, the cats eye can be seen as a visual feature of a cat. This feature is generally consistitent with most cats. 

Feature extraction extracts features in an image and finds a pattern between them and if these features are consistent between all images of an object it will learn that these features can be used to identify the object.

The main feature of feature extraction in image classification is to transform the visual feature to a mathematical vector for computational use.

We do this so computers will be able to compare the similarity of features easily.

### Convolution


## Refrences
[High level outline of CNNs](https://medium.freecodecamp.org/an-intuitive-guide-to-convolutional-neural-networks-260c2de0a050)